In [1]:
include("src/lisa_meta.jl")
include("src/lisa_util.jl")

using ..LisaNeo4j
using ..LisaMeta
using ..Store
using ..Graph
using ..Util

using CSV
using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig
using SHA
using Base64
using DataFrames

hll = SetCore.HllSet{10}()

db_source = SQLite.DB("lisa_enron.db")
# Create an empty HDF5 file. Run it only once, because it will overwrite the file
# h5open("hll_algebra.hdf5", "w") do f
#     # The file is now open, but it's empty
# end

# Connect to the SQLite meta database
db_meta = Graph.DB("hll_algebra.db")

# Define the URL of your Neo4j instance
url = "http://localhost:7474/db/neo4j/tx/commit"
# Define the headers
headers = ["Content-Type" => "application/json", "Authorization" => "Basic " * base64encode("neo4j:neo4j")]

2-element Vector{Pair{String, String}}:
  "Content-Type" => "application/json"
 "Authorization" => "Basic bmVvNGo6bmVvNGo="

In [2]:
# Query the first 10 rows of the table
fields = "\"Message-ID\" AS message_id, Date, \"From\", \"To\", Subject, content, user, labeled"
# df = DBInterface.execute(db_source, "SELECT $fields FROM emails ORDER BY Date LIMIT 10") |> DataFrame

# # Print the DataFrame
# print(df)

"\"Message-ID\" AS message_id, Date, \"From\", \"To\", Subject, content, user, labeled"

In [3]:
# Create list of dates
df_dates = DBInterface.execute(db_source, "SELECT DISTINCT strftime('%Y-%m-%d', Date) AS Date FROM emails ORDER BY Date") |> DataFrame

# Get the first 10 rows of the DataFrame
df_dates_10 = first(df_dates, 10)

# Print the first 10 rows
print(df_dates_10)

10×1 DataFrame
│ Row │ Date       │
│     │ String     │
├─────┼────────────┤
│ 1   │ 1980-01-01 │
│ 2   │ 1986-04-26 │
│ 3   │ 1986-05-01 │
│ 4   │ 1997-01-01 │
│ 5   │ 1997-03-03 │
│ 6   │ 1997-03-05 │
│ 7   │ 1997-03-06 │
│ 8   │ 1997-03-07 │
│ 9   │ 1997-03-11 │
│ 10  │ 1997-03-16 │

In [4]:
strings = Set{String}()
push!(strings, "aba567b86136d0ad7cdaec68e600ca02a3400544") 
push!(strings, "g1a567b86136d0ad7cdaec68e600ca02a3400530")
push!(strings, "b1a567b86136d0ad7cdaec68e600ca02a3400526")

println("sha1_union([A, B, C]): ", Util.sha1_union(strings))
println("sha1_intersection([A, B, C]): ", Util.sha1_intersect(strings))

println("sha1_complement(B, A): ", Util.sha1_comp("g1a567b86136d0ad7cdaec68e600ca02a3400530", "aba567b86136d0ad7cdaec68e600ca02a3400544"))

println("sha1_complement(A, B): ", Util.sha1_comp("aba567b86136d0ad7cdaec68e600ca02a3400544", "g1a567b86136d0ad7cdaec68e600ca02a3400530"))


println("sha1_xor(B, A): ", Util.sha1_xor("g1a567b86136d0ad7cdaec68e600ca02a3400530", "aba567b86136d0ad7cdaec68e600ca02a3400544"))

print("sha1_xor(A, B): ", Util.sha1_xor("aba567b86136d0ad7cdaec68e600ca02a3400544", "g1a567b86136d0ad7cdaec68e600ca02a3400530"))

sha1_union([A, B, C]): c9847d0263de186fc4e3f56b648895565bcd5952
sha1_intersection([A, B, C]): a899d4a28e2c26ae888e964aebec6bfedf30b806
sha1_complement(B, A): e08b92300f215b925bca6dd3719ae76110c5279d
sha1_complement(A, B): ff568d861f973009bffc59bea505aa9305a70b1e
sha1_xor(B, A): 5398bf9f841ab7fcc2e705a017bd1a1113999671
sha1_xor(A, B): 5398bf9f841ab7fcc2e705a017bd1a1113999671

In [5]:
# using Base.BitSet

A = BitSet([1, 2, 3])
B = BitSet([1,2,3])

relative_complement = BitSet(setdiff(B, A))
println("setdiff(B, A): ", relative_complement)

relative_complement = BitSet(setdiff(A, B))
println("setdiff(A, B): ", relative_complement)



setdiff(B, A): BitSet([])
setdiff(A, B): BitSet([])


In [6]:
date = "1980-01-01"
df_day = LisaMeta.get_emails_by_date(db_source, date, fields, 100)

,message_id,Date,From
,String,String,String
1,<12929996.1075855668941.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
2,<29770699.1075855669609.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
3,<13537630.1075855669909.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
4,<27903020.1075855669931.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
5,<17449361.1075855672476.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
6,<5722531.1075855698441.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
7,<12635597.1075855702772.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
8,<16041239.1075855708564.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com
9,<12860336.1075855675177.JavaMail.evans@thyme>,1980-01-01 00:00:00,phillip.allen@enron.com


In [7]:
# Ingest the data into the store
columns_daily = LisaMeta.ingest_df_by_column(db_meta, df_day, "daily")
row_daily = LisaMeta.ingest_df_by_row(db_meta, df_day, "daily"; p=10)

# Commit the data to the store
message = string("Ingested data for ", date)
Store.commit(db_meta, "lisa_enron.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", message, Config())

In [8]:
query = raw"SELECT * FROM nodes WHERE json_extract(props, '$.From') = 'phillip.allen@enron.com'"

refs_col = LisaMeta.select_sha1_by_label(db_meta, "column", "nodes", -1)
println(refs_col)
refs_rows = LisaMeta.select_sha1_by_query(db_meta, query, -1)
println(refs_rows)

refs = union(refs_col, refs_rows)

nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, refs, nodes)
LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)

Set(Any["d0d201c405adf0df65759c16de98f8ef4795d737", "e65cbaf2e457609a0d9d3bb58bcd62a1cd0e73be", "db3170975744a4abd9d401e15876f414f7988bba", "aaf03de4d2597380806e6726f4dc17ce4b32ae74", "935b6035429296997bdfb2505ab644f56c22ccd6", "bc018f6051278ee0732303d5197154dc61a6f167", "8342e461fb2cb55b121fca66cc8ca745ee2f72ae", "6927c490b1aac716b820616b2de8b7191ee9d68d"])
Set(Any["a10143e19099cd2679132f63bf59afd3e275bddf", "1c7335328c96de135cf28a6d9cf5d5e419646692", "49b229f31f463b9d29ff178af9fb13ebbad0f151", "0903525a12d3a22b1538b7a6ecb9e220de17a5cc", "3107f7642141eb26ac2763a320e57c022546da10", "55912dbbd88c6796e195da2abd8e3d4e7f562121", "cb63357c93f9c1c0c2c8fbdd27e3e902cbea1de7", "cdd363110058711a9384dd57c7f92ccd88476aeb", "0d2715b918ae2122cfd24cf9b1bc9eb5af4bf3c5", "69a28c5fff086024832fd012c74b21f6a015553d"])


In [9]:
row_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, refs_rows, row_nodes)

refs = Graph.node_union(db_meta, row_nodes)
println("Union node added: ", refs)

LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
LisaNeo4j.add_neo4j_edges_by_refs(db_meta.sqlitedb, Set(refs), url, headers)


Union node added: Any["a10143e19099cd2679132f63bf59afd3e275bddf", "1c7335328c96de135cf28a6d9cf5d5e419646692", "49b229f31f463b9d29ff178af9fb13ebbad0f151", "0903525a12d3a22b1538b7a6ecb9e220de17a5cc", "3107f7642141eb26ac2763a320e57c022546da10", "55912dbbd88c6796e195da2abd8e3d4e7f562121", "cb63357c93f9c1c0c2c8fbdd27e3e902cbea1de7", "cdd363110058711a9384dd57c7f92ccd88476aeb", "0d2715b918ae2122cfd24cf9b1bc9eb5af4bf3c5", "69a28c5fff086024832fd012c74b21f6a015553d", "a9f2fdbdfd13380abcb686d15ea6eecd937b0b25"]


In [10]:
refs = Set(["d0d201c405adf0df65759c16de98f8ef4795d737", "a10143e19099cd2679132f63bf59afd3e275bddf"])

row_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, refs, row_nodes)
println("Row nodes: ", row_nodes)

refs = Graph.node_intersect(db_meta, row_nodes[1], row_nodes[2])

LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
LisaNeo4j.add_neo4j_edges_by_refs(db_meta.sqlitedb, Set(refs), url, headers)


Row nodes: Any[DataFrameRow
│ Row │ sha1                                     │ labels     │ d_sha1                                   │ dataset                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [11]:
row_ref = Set(["a10143e19099cd2679132f63bf59afd3e275bddf"])
col_refs = LisaMeta.select_sha1_by_label(db_meta, "column", "nodes", -1)

# refs = union(row_ref, col_refs)

row_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, row_ref, row_nodes)

col_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, col_refs, col_nodes)

refs = Graph.projection_1_n(db_meta, row_nodes[1], col_nodes)

LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
LisaNeo4j.add_neo4j_edges_by_refs(db_meta.sqlitedb, Set(refs), url, headers)

In [12]:
row_ref = Set(["1c7335328c96de135cf28a6d9cf5d5e419646692", "49b229f31f463b9d29ff178af9fb13ebbad0f151", "0903525a12d3a22b1538b7a6ecb9e220de17a5cc"])

row_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, row_ref, row_nodes)


refs = Graph.projection_m_n(db_meta, row_nodes, col_nodes)

LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
LisaNeo4j.add_neo4j_edges_by_refs(db_meta.sqlitedb, Set(refs), url, headers)

# References

1. https://github.com/rafaelmartinelli/Knapsacks.jl